# Week 10 - Assignment Pt II

## Credit Approval

* from https://archive.ics.uci.edu/ml/datasets/Credit+Approval

In [1]:
# create seed
import random

# provide your student id as stud_id
stud_id = 12345678
my_seed = random.seed(stud_id)

In [2]:
import string
import random

columns = [''.join(random.choices(string.ascii_lowercase, k=3)) for _ in range(15)]
columns.append('target')
columns

['sqf',
 'gpp',
 'xci',
 'dva',
 'ymu',
 'vsf',
 'olo',
 'gjl',
 'hvz',
 'jsd',
 'fom',
 'wpt',
 'jfc',
 'dku',
 'fhf',
 'target']

## Data Prep

In [3]:
# construct the dataframe
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('week10.db')

crx = pd.read_sql_query('SELECT * FROM features JOIN target USING (id)', conn)
crx.drop('id', axis=1, inplace=True)
crx.columns = columns
crx = crx[np.random.default_rng(seed=my_seed).permutation(crx.columns.values)]
print(crx.shape)
print(crx.head())

(1380, 16)
  jfc    dku dva hvz   gjl wpt    gpp  fhf jsd sqf olo vsf  fom target    xci  \
0   g  00202   u   t  1.25   f  30.83    0   t   b   v   w    1      +  0.000   
1   g  00043   u   t  3.04   f  58.67  560   t   a   h   q    6      +  4.460   
2   g  00280   u   t  1.50   f  24.50  824   f   a   h   q    0      +  0.500   
3   g  00100   u   t  3.75   t  27.83    3   t   b   v   w    5      +  1.540   
4   s  00120   u   t  1.71   f  20.17    0   f   b   v   w    0      +  5.625   

  ymu  
0   g  
1   g  
2   g  
3   g  
4   g  


In [ ]:
# columns with missing values
crx.loc[: , (crx == '?').any()].columns

In [ ]:
# replace ? with np.nan
import numpy as np

crx.replace('?', np.nan, inplace=True)
crx.loc[: , (crx == '?').any()].columns

In [ ]:
crx.info()

In [ ]:
# identify constants


In [ ]:
# identify quasi constant values (sometimes these may be boolean features)


In [ ]:
# identify duplicate rows


In [ ]:
# drop duplicate rows if necessary


In [ ]:
# check of duplicate columns


In [ ]:
# drop the variables that are duplicated or low in variance if needed


In [ ]:
# all of the missing values are objects (strings)
# find how many unique labels are in each feature
for val in crx.columns.sort_values():
    print(val, len(crx[val].dropna().unique()))

In [ ]:
# check nulls for dataframe
crx.isnull().sum()

### Replace NA with Mode

In [ ]:
# replace missing data with mode and confirm isnull
for feat in crx.columns:
    crx[feat].fillna(crx[feat].mode()[0], inplace=True)
    
crx.isnull().sum()

## Exploratory Data Analysis

Dependent variable = target

In [ ]:
# train test split; dependent variable = target


In [ ]:
# get the X_train info


In [ ]:
# describe basic statistics


### Outliers

https://www.projectpro.io/recipes/deal-with-outliers-in-python

* Drop
* Mark
* Rescale

In [ ]:
# print outliers by feature


In [ ]:
# show X_train boxplots


### Scalers

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

* from sklearn.preprocessing import MinMaxScaler
* from sklearn.preprocessing import minmax_scale
* from sklearn.preprocessing import MaxAbsScaler
* from sklearn.preprocessing import StandardScaler
* from sklearn.preprocessing import RobustScaler
* from sklearn.preprocessing import Normalizer
* from sklearn.preprocessing import QuantileTransformer
* from sklearn.preprocessing import PowerTransformer

In [ ]:
# PowerTransformer scaler for outliers


In [ ]:
# confirm scaling with X_train describe


In [ ]:
# confirm scaling with X_train boxplots


In [ ]:
# confirm scaling by printing outliers by feature


### Mapping Dependent Variable

In [ ]:
# y_train value counts


In [ ]:
# map y_train values: - = 0; + = 1


### Bi Label Mapping

In [ ]:
# make a list of categorical column names
X_train.select_dtypes(['object']).columns

In [ ]:
# identify features with bi labels


In [ ]:
# print X_train head


In [ ]:
# bi-label mapping
bi_labels = [...]

for feat in bi_labels:
    print(X_train[feat].value_counts())
    bi0 = X_train[feat].value_counts().index[0]
    bi1 = X_train[feat].value_counts().index[1]
    X_train[feat] = X_train[feat].map({bi0:0,bi1:0})
    X_test[feat] = X_test[feat].map({bi0:0,bi1:0})

X_train.head() # confirms bi label mapping

### One Hot Encoding Alternatives

For features with many labels

* https://medium.com/analytics-vidhya/stop-one-hot-encoding-your-categorical-variables-bbb0fba89809
* https://medium.com/swlh/stop-one-hot-encoding-your-categorical-features-avoid-curse-of-dimensionality-16743c32cea4
* https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02 (frequency and mean encoding)

In [ ]:
# review features with multiple labels
for val in X_train.columns.sort_values():
    print(val, len(X_train[val].dropna().unique()))

In [ ]:
# identify features with more than 5 features and use frequency encoding
freq_feats = [...]

for feat in freq_feats:
    freq = X_train.groupby(feat).size()/len(X_train)
    X_train.loc[:, feat] = X_train[feat].map(freq)
    freq = X_test.groupby(feat).size()/len(X_test)
    X_test.loc[:, feat] = X_test[feat].map(freq)

### One Hot Encoding

In [ ]:
# onehotencode the object dtypes for features with 3 - 5 unique labels
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories='auto', drop='first', sparse=False)

cat_features = [...]
ohe_train = ohe.fit_transform(X_train[cat_features])
ohe_train = pd.DataFrame(ohe_train, columns=ohe.get_feature_names_out(cat_features))
ohe_train.index = X_train.index
# use this line if you get an error
# X_train = pd.DataFrame(ohe_train, columns=ohe.get_feature_names(cat_features))
X_train = X_train.join(ohe_train)
X_train.drop(cat_features, axis=1, inplace=True)

ohe_test = ohe.transform(X_test[cat_features])
ohe_test = pd.DataFrame(ohe_test, columns=ohe.get_feature_names_out(cat_features))
ohe_test.index = X_test.index
X_test = X_test.join(ohe_test)
X_test.drop(cat_features, axis=1, inplace=True)

print(X_train.shape)
print(X_train.head())

In [ ]:
# print X_train info


In [ ]:
# plot.pie to see if target is balanced


In [ ]:
# check for null values in X test


In [ ]:
# build a logistic regression model
# is it balanced? no, use class_weight='balanced'
# is it a small dataset? yes, use liblinear for solver
# print training score
# print test score
# print accuracy score
# print rocauc score
# print confusion matrix


In [ ]:
# what is the recall? (derive the answer from the confusion matrix)
# tn, fp, fn, tp is given in cell above
# for example false negative rate (fnr) = fn / (fn + tp) so to get the fnr type fn / (fn + tp)

In [ ]:
# what is the precision? (derive the answer from the confusion matrix)